In [1]:
import numpy as np
import pandas as pd
import re
import json
import time
import os

from IPython.display import display
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
from collections import Counter

import datetime

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_34960\2444131415.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# loading location infor

In [5]:
df_com = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\out\경기\lat_lng_final.xlsx")
df_com = df_com.drop(["Unnamed: 0"],axis=1)
df_com

,지원센터단위코드,지원센터구분코드,지원센터코드2,lat,lng
0,기초,푸드뱅크,가평군푸드뱅크,127.517915,37.814147
1,광역,푸드뱅크,경기도광역푸드뱅크,126.971821,37.299404
2,기초,푸드뱅크,고양시문촌7푸드뱅크,126.756267,37.674637
3,기초,푸드뱅크,고양시문촌9푸드뱅크,126.760809,37.672214
4,기초,푸드마켓,고양시한아름푸드마켓,126.780077,37.646015
...,...,...,...,...,...
79,기초,푸드뱅크,안산나눔푸드뱅크,126.818823,37.313972
80,기초,푸드뱅크,안산푸드뱅크,126.836962,37.329974
81,기초,푸드뱅크,용인시여럿이함꼐푸드뱅크,127.215592,37.276472
82,기초,Both,의정부푸드뱅크마켓,127.087729,37.737405


# Merging

In [9]:
def change_col(df_):
    if len(df_.columns) ==5:
        col_ori = list(df_.columns)
        col_after = ["모집일자","종류","품목","수량","가액"]
        col_dic = {}
        for i in range(len(col_ori)):
            key_ = col_ori[i]
            value_ = col_after[i]
            col_dic[key_] = value_
        df_ = df_.rename(columns=col_dic)
    else:
        col_ori = list(df_.columns)
        col_after = ["모집일자","이용자 구분","종류","품목","수량","가액"]
        col_dic = {}
        for i in range(len(col_ori)):
            key_ = col_ori[i]
            value_ = col_after[i]
            col_dic[key_] = value_
        df_ = df_.rename(columns=col_dic)
    return df_

In [10]:
def search_index(item_,df_cate):
    return list(df_cate[df_cate["item"]==item_]["index"])[0]

def groupby_machine(df_1):
    date_ = [i for i in list(Counter(df_1["모집일자"]).keys()) if type(i) == datetime.datetime]
    df_3 = pd.DataFrame()
    for da_ in date_:
        df_2 = df_1[df_1["모집일자"]==da_][["모집일자","품목","수량"]]
        df_2["수량"] = df_2["수량"].map(lambda x: int(x))
        df_2 = df_2.groupby("품목").sum(numeric_only=True)
        df_2["모집일자"] = da_
        df_2.reset_index(inplace=True)
        #display(df_2)
        df_2["index"] = df_2["품목"].map(lambda x: search_index(x,df_cate))
        df_2 = df_2[df_2["index"] !=0]
        df_2 = df_2[["모집일자","품목","index","수량"]]
        df_3 = pd.concat([df_3,df_2])
    return df_3

In [17]:
def transform_dataset(DF_all):
    DF_out = pd.DataFrame(columns=["모집일자",1,2,3,4,5,6,7,8,9,10,
                                  11,12,13,14,15,16,17,18,19,20])
    date_2 = list(Counter(DF_all["모집일자"]).keys())
    for da_2 in date_2:
        df_tar = DF_all[DF_all["모집일자"]==da_2]
        df_dic = [ [col["index"],col["수량"]] for row,col in df_tar.iterrows()]
        in_index = len(DF_out)
        First = [0 for i in range(21)]
        DF_out.loc[in_index] = First
        DF_out.loc[in_index,"모집일자"] = da_2
        for index_ in df_dic:
            DF_out.loc[in_index,index_[0]] = index_[1]
    return DF_out

In [19]:
def transfrom_dataset_cum(DF_out):
    DF_cum = DF_out.iloc[:,[0]]
    DF_cum = pd.concat([DF_cum,DF_out.iloc[:,1:].cumsum()],axis=1)
    return DF_cum

In [24]:
def merging_location(df_,df_com,test_):
    df_["lat"] = df_com[df_com["지원센터코드2"]==test_]["lat"].values[0]
    df_["lng"] = df_com[df_com["지원센터코드2"]==test_]["lat"].values[0]
    df_["name"] = test_
    return df_

In [12]:
df_cate = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\경기\2022\out\cate.xlsx")
df_cate = df_cate.rename(columns={0:"item","Unnamed: 1":"index"})
df_cate

,item,index
0,찰기장 40kg,2
1,Vanilla Raspberry Cake KR 245g(바닐라라즈베리케익)1pac...,9
2,y트리오 1kg,0
3,건아로니아,6
4,계란(30구),4
...,...,...
6199,흰색 반팔티셔츠,0
6200,흰설탕,14
6201,"흰우유, 서울우유",3
6202,흰우유/개,3


In [31]:
list_ = os.listdir(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\경기\2022")
list_all = [ i.replace(".xlsx","")   for i in list_ if "xlsx" in i]
dir_path = r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\경기\2022\{}.xlsx"

In [30]:
test_ = list_all[0]
test_

'가평군푸드뱅크'

In [14]:

df_supply = pd.read_excel(dir_path.format(test_),sheet_name="기부물품 제공")
df_demand =  pd.read_excel(dir_path.format(test_),sheet_name="기부물품 모집")

In [15]:
df_supply = change_col(df_supply)
df_demand = change_col(df_demand)

In [16]:
# groupby 모집일자 
DF_D = groupby_machine(df_demand)
DF_S = groupby_machine(df_supply)
DF_S["수량"] = DF_S["수량"].map(lambda x: -1*x )
DF_all = pd.concat([DF_S,DF_D])
DF_all = DF_all[["모집일자","index","수량"]].groupby(["모집일자","index"]).sum().reset_index()

In [25]:
# transforming dataset into vector-related format
DF_out = transform_dataset(DF_all)
DF_cum = transfrom_dataset_cum(DF_out)

In [28]:
# adding location infor
DF_out = merging_location(DF_out,df_com,test_)
DF_cum = merging_location(DF_cum,df_com,test_)

# Combine all function

In [53]:
dir_path = r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\경기\2022\{}.xlsx"

def Preprocessing_machine(dir_path,test_):
    df_supply = pd.read_excel(dir_path.format(test_),sheet_name="기부물품 제공")
    df_demand =  pd.read_excel(dir_path.format(test_),sheet_name="기부물품 모집")
    df_supply = change_col(df_supply)
    df_demand = change_col(df_demand)
    # groupby 모집일자 
    DF_D = groupby_machine(df_demand)
    DF_S = groupby_machine(df_supply)
    DF_S["수량"] = DF_S["수량"].map(lambda x: -1*x )
    DF_all = pd.concat([DF_S,DF_D])
    DF_all = DF_all[["모집일자","index","수량"]].groupby(["모집일자","index"]).sum().reset_index()
    # transforming dataset into vector-related format
    DF_out = transform_dataset(DF_all)
    DF_cum = transfrom_dataset_cum(DF_out)
    # adding location infor
    DF_out = merging_location(DF_out,df_com,test_)
    DF_cum = merging_location(DF_cum,df_com,test_)
    return DF_out,DF_cum

In [54]:
Out_1 = pd.DataFrame()
Out_2 = pd.DataFrame()
tq = tqdm(list_all)
for test_ in tq:
    print(test_)
    ou_1,ou_2 = Preprocessing_machine(dir_path,test_)
    Out_1 = pd.concat([Out_1,ou_1])
    Out_2 = pd.concat([Out_2,ou_2])
    path_2 = r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\경기\2022\out\{}.xlsx"
    ou_1.to_excel(path_2.format(test_))
    ou_2.to_excel(path_2.format(test_+"_cum"))

  0%|          | 0/84 [00:00<?, ?it/s]

가평군푸드뱅크
경기도광역푸드뱅크
고양시문촌7푸드뱅크
고양시문촌9푸드뱅크
고양시한아름푸드마켓
고양시흰돌기초푸드뱅크
광명시푸드뱅크마켓센터
광주시중앙푸드뱅크&푸드마켓
광주시참사랑푸드뱅크
구리사랑나눔푸드뱅크
구리시푸드뱅크
군포기초푸드뱅크
김포시푸드뱅크
김포한강푸드뱅크
나누리푸드뱅크
남양주푸드뱅크
늘만나푸드뱅크
덕양행신기초푸드뱅크
동두천천사푸드뱅크
문화협회푸드뱅크
복사골푸드뱅크
봉담아리푸드뱅크
부천기초푸드뱅크
부천시다사랑푸드마켓
부천행복한동행푸드뱅크
비전푸드뱅크
사랑나눔이천푸드마켓
사랑푸드뱅크
새희망푸드뱅크
서안성푸드뱅크
서평택푸드뱅크
성남시청솔푸드뱅크
성남열린푸드뱅크마켓
소망더함푸드뱅크
솔모루푸드뱅크
수원시장안푸드뱅크
수원시푸드뱅크
수원시해누리푸드마켓
시흥푸드뱅크마켓센터
아름다운동행 푸드뱅크
안산나눔푸드뱅크
안산사랑푸드뱅크
안산시광림푸드뱅크
안산푸드뱅크
안성동부푸드뱅크
안성시푸드뱅크
안성연화마을푸드뱅크
안양시유쾌한푸드뱅크
안양시푸드뱅크
안양착한푸드뱅크마켓
양주기초푸드뱅크
양주연화푸드마켓
양평군기초푸드뱅크
여주구세군푸드뱅크
여주군좋은이웃기초푸드뱅크
여주기초푸드뱅크
연천군푸드뱅크
영통푸드뱅크
오산시푸드뱅크
용인사랑나눔푸드뱅크
용인시여럿이함꼐푸드뱅크
웃어라나눔과기쁨푸드뱅크
의왕시푸드뱅크
의정부푸드뱅크마켓
이천시선양푸드뱅크
이천시푸드뱅크
파주시사랑나눔푸드뱅크
파주시희망나눔푸드뱅크
파주시희망푸드마켓
팔도푸드뱅크
평택시송탄기초푸드뱅크
평택시푸드마켓1호점
평택푸드마켓2호점
평택푸드뱅크
포천푸드뱅크
하남시푸드뱅크
행복나눔푸드뱅크
화성시나래울푸드마켓
화성시행복나눔푸드마켓
화성시행복나눔푸드뱅크
화성여럿이함께푸드뱅크
화성은혜푸드뱅크
희망나눔푸드뱅크
희망등대푸드뱅크


In [56]:
Out_1 = Out_1.reset_index(drop=True)
Out_2 = Out_2.reset_index(drop=True)

In [58]:
Out_1.to_excel(path_2.format("Total"))
Out_2.to_excel(path_2.format("Tota_cuml"))

In [59]:
df_com

,지원센터단위코드,지원센터구분코드,지원센터코드2,lat,lng
0,기초,푸드뱅크,가평군푸드뱅크,127.517915,37.814147
1,광역,푸드뱅크,경기도광역푸드뱅크,126.971821,37.299404
2,기초,푸드뱅크,고양시문촌7푸드뱅크,126.756267,37.674637
3,기초,푸드뱅크,고양시문촌9푸드뱅크,126.760809,37.672214
4,기초,푸드마켓,고양시한아름푸드마켓,126.780077,37.646015
...,...,...,...,...,...
79,기초,푸드뱅크,안산나눔푸드뱅크,126.818823,37.313972
80,기초,푸드뱅크,안산푸드뱅크,126.836962,37.329974
81,기초,푸드뱅크,용인시여럿이함꼐푸드뱅크,127.215592,37.276472
82,기초,Both,의정부푸드뱅크마켓,127.087729,37.737405
